<h1 align="center"> Airline Delays Toy Example </h1>
<h2 align="center"> W261 - Final Project </h2>
<h5 align="center"> by Team 25: Adam Sohn, Chandra Shekar Bikkanur, Jayesh Parikh, Tucker Anderson</h5>

In [2]:
import pandas as pd
import numpy as np

### Introduction

In this example we will go through a few steps of the parallelized linear regression model. Our model sought to provide a predictive system with which flight travelers could input several features of the flight at arrival, and the model would output the predicted flight delay at the destination in minutes. In our research, we found that we achieved best regression model (by \\(MSE\\) and \\(R^2\\) through spark's mllib.regression.LinearRegression model class. This class provides an interface to train an OLS model with optional parameters (regularization, standardization etc.) based on our features.

### What is parallelized multivariate OLS regression?

In Ordinary Least Squares (OLS) regression, we are attempting to minimize the squared difference between the predicted output of our model and the actual observed values of our dependent variable (in this case flight arrival delay). Here is our loss function, where \\(\theta\\) contains our model weights (one for each variable, plus one bias term), \\(x_i\\) contains independent variable values and \\(\\) over \\(n\\) records.

\\(f(\theta) = \frac{1}{n} * \sum_{i=1}^n \left[ \boldsymbol{\theta}^T\cdot\mathbf{x}'\_i - y\_i\right]^2\\)

And through the parital derivative of this loss function with respect to \\(\theta\\):

\\(f'(\theta) = \frac{2}{n}\,\sum\_{i=1}^{n}\left[ \boldsymbol{\theta}^T\cdot\mathbf{x}'\_i - y_i\right] \cdot \mathbf{x}'\_i\\)

Algebraically, this gives a closed form solution to minimize our loss function (by setting the gradient to 0):

\\(\theta^* = (X^TX)^{-1}X^Ty\\)

However, this solution is computationally intractable as \\(n\\) increases (\\(O(n^3)\\) time complexity!), and cannot be easily parallelized. The large inverted matrices and associated products cannot be served to individual partitions.

In Spark, the mllib.regression implementations use various Gradient Descent methodologies to produce parallelizable approaches to estimate the minimum gradient given our list of features and weights. In gradient descent, we descend in our model parameter space in each training cycle or epoch through the following formula:

\\(\theta\_{\text{new}} = \theta\_{\text{old}} - \eta \cdot \nabla\_{\theta} f(\theta) \\)

Where \\(\eta\\) is a configurable hyperparameter for Gradient Descent learning rate. If properly tuned, we guarantee we will converge to at least a local minimum in our parameter space.
This concession to the closed form solution means that we don't have a guarantee of global minimum, but we now can parallelize solutions by sending each record to isolated partitions and calculating their assocaited new gradient compenent, then finally reducing down to a total loss calculation. Each iteration of this map-reduce operation will eventually give us a minimized loss function, in much quicker time than the closed form solution.

### So what's going on in our model?

Our model uses various flight features as independent variables, with "arrival delay" as our dependent \\(y\\) value. For this toy example, we will take a sub-section of them, namely "DEP_DELAY" and "CRS_ELAPSED_TIME". "DEP_DELAY" is simply the delta in minutes between the each flight was expected to depart the origin airport and when it actually left where "CRS_ELAPSED_TIME" represents the expected length of the flight in minutes.

We are also utilizing an elastic net regularization term (combination of L1 and L2 regularization), defined here (just a combination of L1 and L2 regression):

\\(c = \lambda((1-\alpha\sum_1^{m}\theta^2) + (\alpha \sum^m_1 |\theta|))\\)

Where our cross validation deemed that a value of \\(\alpha\\) of 0.5 and a regularization term \\(\lambda\\) of 0.1 produces the best results. Combining our gradient descent approach from above with this regularization term, we get this loss function:

\\(f(\theta) = \frac{1}{n} * \sum_{i=1}^n \left[ \boldsymbol{\theta}^T\cdot\mathbf{x}'\_i - y\_i\right]^2 + ((0.5\sum_1^{m}\theta^2) + (0.5\sum^m_1 |\theta|))\\)

And taking the loss function's derivate to compute the gradient:

\\(f'(\theta) = \frac{2}{n}\,\sum\_{i=1}^{n}\left[ \boldsymbol{\theta}^T\cdot\mathbf{x}'\_i - y_i\right] \cdot \mathbf{x}'\_i + 0.5 sign(\boldsymbol{\theta}) + 0.5\cdot \theta \\)

Now we will go through a few iterations of gradient descent, step-by-step, without any feature scaling to simplify calculations.

In [9]:
FEATURE_COLUMNS = ["INTERCEPT", "DEP_DELAY", "CRS_ELAPSED_TIME", ]
M = len(FEATURE_COLUMNS)
OUTPUT_COLUMNS = ["xj_input", "y_output", "y_predicted", "regularization", "xj_gradient"]
theta = np.array([[0, 0, 0]])
LAMBDA = 1

def gradientDesc(theta):

  #100 mile flight dep delayed 0 minutes going North planned to take ten minutes
  REC_1 = np.array([1, 0, 10])
  #50 mile flight dep delayed 0 minutes going East planned to take 5 minutes
  REC_2 = np.array([1, 0, 5])
  #100 mile flight dep delayed 5 minutes going South planned to take 10 minutes
  REC_3 = np.array([1, 5, 10])

  df = pd.DataFrame([REC_1,
                     REC_2,
                     REC_3],
                     columns=FEATURE_COLUMNS)

  x1 = np.array([df.iloc[0]])
  x2 = np.array([df.iloc[1]])
  x3 = np.array([df.iloc[2]])

  #flight delays for each flight
  y1 = 10
  y2 = 15
  y3 = 20

  #regularization term with alpha = 0.5
  reg = ((.5*(theta)) + (.5*(np.sign(theta))))

  #predicted values and calculating gradients
  ######EACH CALCULATION CAN BE PARALLELIZED#######
  y1_pred = np.dot(theta, x1.T)
  loss1 = np.sum(((y1_pred - y1) * x1)**2) + .25*np.sum((theta**2)) + .5*np.sum((abs(theta)))  
  grade1 = ((y1_pred - y1) * x1) + reg

  y2_pred = np.dot(theta, x2.T)
  loss2 = np.sum(((y2_pred - y2) * x2)**2) + .25*np.sum((theta**2)) + .5*np.sum((abs(theta)))  
  grade2 = ((y2_pred - y2) * x2) + reg

  y3_pred = np.dot(theta, x3.T)
  loss3 = np.sum(((y1_pred - y1) * x1)**2) + .25*np.sum(.5*(theta**2)) + .5*np.sum((abs(theta)))  
  grade3 = ((y3_pred - y3) * x3) + reg
  
  ######REDUCER REQUIRED HERE WHEN PARALLELIZED######
  grade_all = (grade1 + grade2 + grade3)* (2/3)
  loss_all = (loss1 + loss2 + loss3) /3 

  #update weights with learning rate of 0.01
  new_theta = theta - .01*(grade_all)
  
  return(new_theta, loss_all)

# now to conduct a few gradient descents over our dataset (batch gradient descent), each time updating the weights of our model
for i in range(101):
  theta,loss = gradientDesc(theta)
  if (i%10 == 0):
    print("Gradient update step", i)
    print("Loss for step", i, ":", loss)
    print("Current weights for step", i, theta)

Gradient update step 0
Loss for step 0 : 8683.333333333334
Current weights for step 0 [[0.3 0.66666667 2.5 ]]
Gradient update step 10
Loss for step 10 : 1930.1072761081157
Current weights for step 10 [[0.3275053 0.8301659 1.44084145]]
Gradient update step 20
Loss for step 20 : 1773.9093994291368
Current weights for step 20 [[0.459487 0.96992319 1.38639657]]
Gradient update step 30
Loss for step 30 : 1689.1613432988604
Current weights for step 30 [[0.58288983 1.02320053 1.36013942]]
Gradient update step 40
Loss for step 40 : 1648.3242476502955
Current weights for step 40 [[0.69479496 1.04490939 1.34263967]]
Gradient update step 50
Loss for step 50 : 1623.918140578744
Current weights for step 50 [[0.79529423 1.05510622 1.32911503]]
Gradient update step 60
Loss for step 60 : 1606.3775235723504
Current weights for step 60 [[0.88520927 1.06094798 1.3177879 ]]
Gradient update step 70
Loss for step 70 : 1592.2481166344912
Current weights for step 70 [[0.96553451 1.06500458 1.30794268]]
Gradient update step 80
Loss for step 80 : 1580.2031376229809
Current weights for step 80 [[1.03725014 1.06821417 1.29924983]]
Gradient update step 90
Loss for step 90 : 1569.674607219009
Current weights for step 90 [[1.10126386 1.07093276 1.29152503]]
Gradient update step 100
Loss for step 100 : 1560.37365570617
Current weights for step 100 [[1.15839746 1.07330723 1.28464272]]

We see that our simple model weights with made up parameter values converges to the weights displayed above using our batch Gradient Descent approach.

Note that this example utilizes batch Gradient Descent and not stochastic or mini-batch Gradient Descent. This means we are calculating the gradient utilizing the whole dataset at once. We chose to stick with this methodology instead of reverting to an SGD approach as our linear model converged relatively quickly utilizinig batch Gradient Descent.

#### Summary
The hand-calculated Gradient Descent successfully reduced loss with each successive step, indicating that the process is effective for finding a set of weights that will return a minimal squared-loss value. 

Regarding parallelization, Spark has the opportunity to parallelize by:
* mapping (prediction, loss, gradient) for each row to an assigned worker.
* combining (grade_all, loss_all) is an optional step for all rows.
* reduction (grade_all, loss_all) for all rows on a reducer.